<h1>Geopotential Perturbation Calculations</h1>

<p> Geopotential height perturbations refer to deviations or changes in the geopotential height at a particular location in the Earth's atmosphere, caused by various atmospheric processes such as convection, advection, and atmospheric waves. Geopotential height is a measure of the potential energy per unit mass of a particle at a particular location in the atmosphere relative to a reference level, usually the Earth's surface.</p>


In [3]:
import numpy as np
import xarray as xr

# Calculate climatorlogy
flag = False
if flag:
    # Load the ERA5 data
    ds_climatology = xr.open_dataset('/Users/joaojesus/Desktop/final_year_proj/samples/2023-04-18-13-25-18.nc')

    # Subset the data to the desired sub-region and pressure levels
    ds_sub = ds_climatology.sel(latitude=slice(90,30), longitude=slice(-180,180), level=[1,5,10,50,100,200,500])

    print('aggregating data')
    # Group the data by the day of the year and compute the mean value for each day
    climatology = ds_sub.groupby('time.dayofyear').mean(dim='time')

    print('saving data')
    climatology.to_netcdf('climatology_geopotential_era5_dayofyear.nc')


In [7]:
climatology = xr.open_dataset('/Users/joaojesus/Desktop/final_year_proj/climatology_geopotential_era5_dayofyear.nc')
climatology = climatology.sel(latitude=slice(80,45), longitude=slice(-180,180), level=[1,5,10,50,100,200,500])


In [6]:
climatology

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 201, level: 7, dayofyear: 184)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 80.0 79.75 79.5 79.25 ... 30.5 30.25 30.0
  * level      (level) int32 1 5 10 50 100 200 500
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366
Data variables:
    z          (dayofyear, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-04-18 12:43:40 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [19]:
# calculate data
ds = xr.open_dataset('/Users/joaojesus/Desktop/final_year_proj/samples/geopotential_2009_2013_2018.nc')


In [20]:
# select onlu the year 2009
ds = ds.sel(time=slice('2018-01-01','2018-03-31'))

In [21]:
ds_sub = ds.sel(latitude=slice(80,30), longitude=slice(-180,180), level=[1,5,10,50,100,200,500])


In [22]:
# Loop over each day and time and calculate the geopotential height perturbations
perturbations = []
for day_time in ds_sub.time:
    day_of_year = day_time.dt.dayofyear
    clim = climatology.sel(dayofyear=day_of_year)
    clim_broadcast = clim.broadcast_like(ds_sub.sel(time=day_time))
    pert = (ds_sub.sel(time=day_time)['z'] - clim_broadcast['z']) / 9.80665  # Divide by g to convert to units of meters
    perturbations.append(pert)
# Concatenate the perturbation data into a single array
perturbations = xr.concat(perturbations, dim='time')

# Save the perturbation data to a NetCDF file
perturbations.to_netcdf('geopotential_perturbation_data_2018.nc')